In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text
import os

In [2]:
conn = sqlite3.connect(r"C:\Users\dhanu\OneDrive\Desktop\Repo\VR_Training\module_2\database\VR.db")

In [3]:
employees_df = pd.read_sql("SELECT * FROM employees", conn)
departments_df = pd.read_sql("SELECT * FROM departments", conn)
projects_df = pd.read_sql("SELECT * FROM projects", conn)
employee_projects_df = pd.read_sql("SELECT * FROM employee_projects", conn)

In [5]:
employees_df.columns

Index(['employee_id', 'first_name', 'last_name', 'email', 'hire_date',
       'salary', 'department_id', 'manager_id'],
      dtype='object')

In [6]:
departments_df.columns

Index(['department_id', 'department_name', 'location', 'budget'], dtype='object')

In [11]:
pd.merge(employees_df,departments_df,on='department_id',how='right')

,employee_id,first_name,last_name,email,hire_date,salary,department_id,manager_id,department_name,location,budget
0,1,John,Smith,john.smith@company.com,2020-01-15,95000,1,NaN,Engineering,San Francisco,2000000
1,2,Sarah,Johnson,sarah.johnson@company.com,2019-03-22,87000,1,1.0,Engineering,San Francisco,2000000
2,3,Mike,Brown,mike.brown@company.com,2021-06-10,72000,1,1.0,Engineering,San Francisco,2000000
3,10,Jessica,Lee,jessica.lee@company.com,2021-12-03,71000,1,1.0,Engineering,San Francisco,2000000
4,4,Emily,Davis,emily.davis@company.com,2020-09-05,68000,2,NaN,Marketing,New York,800000
5,5,David,Wilson,david.wilson@company.com,2018-11-30,78000,2,4.0,Marketing,New York,800000
6,6,Lisa,Anderson,lisa.anderson@company.com,2022-02-14,85000,3,NaN,Sales,Chicago,1200000
7,7,Tom,Taylor,tom.taylor@company.com,2021-08-20,65000,3,6.0,Sales,Chicago,1200000
8,8,Anna,Martinez,anna.martinez@company.com,2020-04-12,62000,4,NaN,HR,Austin,500000
9,9,Chris,Garcia,chris.garcia@company.com,2019-07-08,89000,5,NaN,Finance,Boston,600000


In [16]:
left_join_res_df = employees_df.merge(departments_df,on="department_id",how='left')[["first_name","last_name","salary","department_name","location",'employee_id']]

In [29]:
employee_count = employees_df.groupby('department_id').count()['employee_id']

In [31]:
employee_count

department_id
1    4
2    2
3    2
4    1
5    1
Name: employee_id, dtype: int64

In [33]:
departments_df['number_of_employee'] = departments_df['department_id'].map(employee_count)

In [35]:
left_join_res_df

,first_name,last_name,salary,department_name,location,employee_id
0,John,Smith,95000,Engineering,San Francisco,1
1,Sarah,Johnson,87000,Engineering,San Francisco,2
2,Mike,Brown,72000,Engineering,San Francisco,3
3,Emily,Davis,68000,Marketing,New York,4
4,David,Wilson,78000,Marketing,New York,5
5,Lisa,Anderson,85000,Sales,Chicago,6
6,Tom,Taylor,65000,Sales,Chicago,7
7,Anna,Martinez,62000,HR,Austin,8
8,Chris,Garcia,89000,Finance,Boston,9
9,Jessica,Lee,71000,Engineering,San Francisco,10


In [21]:
left_join_res_df[~left_join_res_df['employee_id'].isin(inner_join_res_df['employee_id'])]

,first_name,last_name,salary,department_name,location,employee_id


In [18]:
inner_join_res_df = employees_df.merge(departments_df,on="department_id",how='inner')[["first_name","last_name","salary","department_name","location",'employee_id']]

In [5]:
query1 = (employees_df
    .merge(departments_df, on="department_id", how="inner")
)[["first_name","last_name","salary","department_name","location"]]


In [42]:
query2 = (employees_df
    .merge(departments_df, on="department_id", how="inner",)
    .merge(employee_projects_df, on="employee_id", how="inner")
    .merge(projects_df, on="project_id", how="inner")
)

query2

,employee_id,first_name,last_name,email,hire_date,salary,department_id_x,manager_id,department_name,location,budget_x,number_of_employee,project_id,role,hours_allocated,project_name,start_date,end_date,budget_y,department_id_y
0,1,John,Smith,john.smith@company.com,2020-01-15,95000,1,NaN,Engineering,San Francisco,2000000,4,1,Project Manager,40,Mobile App Development,2023-01-01,2023-12-31,500000,1
1,2,Sarah,Johnson,sarah.johnson@company.com,2019-03-22,87000,1,1.0,Engineering,San Francisco,2000000,4,1,Senior Developer,35,Mobile App Development,2023-01-01,2023-12-31,500000,1
2,2,Sarah,Johnson,sarah.johnson@company.com,2019-03-22,87000,1,1.0,Engineering,San Francisco,2000000,4,2,Lead Developer,25,Website Redesign,2023-03-15,2023-09-30,150000,1
3,3,Mike,Brown,mike.brown@company.com,2021-06-10,72000,1,1.0,Engineering,San Francisco,2000000,4,1,Developer,40,Mobile App Development,2023-01-01,2023-12-31,500000,1
4,3,Mike,Brown,mike.brown@company.com,2021-06-10,72000,1,1.0,Engineering,San Francisco,2000000,4,2,Developer,20,Website Redesign,2023-03-15,2023-09-30,150000,1
5,4,Emily,Davis,emily.davis@company.com,2020-09-05,68000,2,NaN,Marketing,New York,800000,2,3,Campaign Manager,40,Marketing Campaign Q2,2023-04-01,2023-06-30,200000,2
6,5,David,Wilson,david.wilson@company.com,2018-11-30,78000,2,4.0,Marketing,New York,800000,2,3,Marketing Specialist,35,Marketing Campaign Q2,2023-04-01,2023-06-30,200000,2
7,6,Lisa,Anderson,lisa.anderson@company.com,2022-02-14,85000,3,NaN,Sales,Chicago,1200000,2,4,Training Coordinator,30,Sales Training Program,2023-02-01,2023-05-31,75000,3
8,7,Tom,Taylor,tom.taylor@company.com,2021-08-20,65000,3,6.0,Sales,Chicago,1200000,2,4,Sales Trainer,25,Sales Training Program,2023-02-01,2023-05-31,75000,3
9,8,Anna,Martinez,anna.martinez@company.com,2020-04-12,62000,4,NaN,HR,Austin,500000,1,5,Project Lead,35,HR System Upgrade,2023-06-01,2023-11-30,100000,4


In [43]:
query2 = (employees_df
    .merge(departments_df, on="department_id", how="inner")
    .merge(employee_projects_df, on="employee_id", how="inner")
    .merge(projects_df, on="project_id", how="inner")
)

query2["budget_utilization"] = (query2["budget_y"] / query2["budget_x"]) * 100

final = query2.rename(columns={"budget_y":"project_budget", "budget_x":"depart_budget"})[
    ["first_name","last_name","department_name","project_name",
     "role","hours_allocated","project_budget","depart_budget","budget_utilization"]
]
final

,first_name,last_name,department_name,project_name,role,hours_allocated,project_budget,depart_budget,budget_utilization
0,John,Smith,Engineering,Mobile App Development,Project Manager,40,500000,2000000,25.00
1,Sarah,Johnson,Engineering,Mobile App Development,Senior Developer,35,500000,2000000,25.00
2,Sarah,Johnson,Engineering,Website Redesign,Lead Developer,25,150000,2000000,7.50
3,Mike,Brown,Engineering,Mobile App Development,Developer,40,500000,2000000,25.00
4,Mike,Brown,Engineering,Website Redesign,Developer,20,150000,2000000,7.50
5,Emily,Davis,Marketing,Marketing Campaign Q2,Campaign Manager,40,200000,800000,25.00
6,David,Wilson,Marketing,Marketing Campaign Q2,Marketing Specialist,35,200000,800000,25.00
7,Lisa,Anderson,Sales,Sales Training Program,Training Coordinator,30,75000,1200000,6.25
8,Tom,Taylor,Sales,Sales Training Program,Sales Trainer,25,75000,1200000,6.25
9,Anna,Martinez,HR,HR System Upgrade,Project Lead,35,100000,500000,20.00


In [22]:
left =  [1,2,3,4,5]
inner = [3,4]
for i in left:
    if i not in inner:
        print(i)

1
2
5


In [7]:
result = (employees_df
    .merge(departments_df, on="department_id", how="inner")
    .loc[lambda df: df["salary"] > 75000]
    .sort_values("salary", ascending=False)
)[["first_name", "last_name", "salary", "department_name"]]


In [8]:
result = employees_df.merge(
    departments_df,
    on="department_id",
    how="left"
)[["first_name", "last_name", "salary", "department_name", "location"]]


In [47]:
rdf = employees_df.merge(employee_projects_df, on="employee_id", how="left")
#.loc[lambda df: df["project_id"].isna()]

In [50]:
rdf.loc[rdf['project_id'].isna()]

,employee_id,first_name,last_name,email,hire_date,salary,department_id,manager_id,project_id,role,hours_allocated
10,9,Chris,Garcia,chris.garcia@company.com,2019-07-08,89000,5,NaN,NaN,NaN,NaN


In [9]:
result = (employees_df
    .merge(employee_projects_df, on="employee_id", how="left")
    .loc[lambda df: df["project_id"].isna()]
)[["first_name", "last_name", "department_id", "project_id"]]


In [52]:
tdf  =departments_df.merge(employees_df, on="department_id", how="left")

In [54]:
gres = tdf.groupby(["department_id", "department_name"], as_index=False)

In [ ]:
gres.agg(employee_count=("employee_id","count"),
         avg_salary=("salary","mean"))

,department_id,department_name,employee_count,avg_salary
0,1,Engineering,4,81250.0
1,2,Marketing,2,73000.0
2,3,Sales,2,75000.0
3,4,HR,1,62000.0
4,5,Finance,1,89000.0


: 

In [ ]:
result = (departments_df
    .merge(employees_df, on="department_id", how="left")
    .groupby(["department_id", "department_name"], as_index=False)
   .agg(employee_count=("employee_id","count"),
         avg_salary=("salary","mean")) 
)


In [11]:
result = departments_df.merge(
    employees_df,
    on="department_id",
    how="left"
)[["first_name", "last_name", "department_name"]]


In [12]:
result = employees_df.merge(
    departments_df,
    on="department_id",
    how="outer"
)[["first_name", "last_name", "department_name"]]


In [13]:
result = employees_df.merge(departments_df, how="cross")[["first_name","last_name","department_name"]].head(20)


In [14]:
result = employees_df.merge(
    employees_df,
    left_on="manager_id",
    right_on="employee_id",
    suffixes=("_emp", "_mgr"),
    how="left"
)

final = result.assign(
    employee_name = result["first_name_emp"] + " " + result["last_name_emp"],
    manager_name  = result["first_name_mgr"].fillna("") + " " + result["last_name_mgr"].fillna("")
)[["employee_name","salary_emp","manager_name","salary_mgr"]]
